# A very basic example
This notebooks shows the basic mechanics of the package on a toy model with a few variables

In [1]:
using Revise, BeliefPropagation, BeliefPropagation.FactorGraphs

Construct a small factor graph from an adjacency matrix: rows correspond to factors, columns to variables. $A_{a, i}$ is one if variable $i$ is involved in factor $a$, zero otherwise.
Here we have 3 factors and 4 variables.

In [2]:
A = [0 1 1 0;
     1 0 1 0;
     0 0 1 1]
g = FactorGraph(A)

FactorGraph{Int64} with 3 factors, 4 variables and 6 edges


Take a look at the factor graph

(This cell is commented out by default because loading Plots for the first time can take a few seconds)

In [3]:
# using Plots, GraphRecipes
# plot(g, size=(400,300), shownames=true)

Specify the number of states (values) that each variable can take

In [4]:
states = [3, 2, 2, 4];

In [5]:
lms(x) = BeliefPropagation.MS(log(x))

lms (generic function with 1 method)

Construct the factors by explicitly providing the values taken for every combination of the inputs

In [6]:
ψ₁ = BPFactor(lms.([1.1 0.3;
               0.5 2.5]))
ψ₂ = BPFactor(lms.([1.0 0.3;
               2.3 1.0;
               0.0 1.2]))
ψ₃ = BPFactor(lms.([0.8 0.0 0.1 0.9;
               0.0 2.9 0.7 1.1]))
ψ = [ψ₁, ψ₂, ψ₃];

Initialize an instance of the `BP` type

In [7]:
const MSF = BeliefPropagation.MS{Float64}
bp = BP(g, ψ, fill(UniformFactor{MSF}(), nvariables(g)),
    [fill(one(MSF), states[dst(e)]) for e in edges(g)],
    [fill(one(MSF), states[dst(e)]) for e in edges(g)],
    [fill(one(MSF), states[i]) for i in variables(g)])

BP{TabulatedBPFactor{BeliefPropagation.MS{Float64}, 2}, UniformFactor{BeliefPropagation.MS{Float64}}, Vector{BeliefPropagation.MS{Float64}}, Vector{BeliefPropagation.MS{Float64}}, FactorGraph{Int64}}(FactorGraph{Int64} with 3 factors, 4 variables and 6 edges
, TabulatedBPFactor{BeliefPropagation.MS{Float64}, 2}[TabulatedBPFactor{BeliefPropagation.MS{Float64}, 2}(BeliefPropagation.MS{Float64}[BeliefPropagation.MS{Float64}(0.09531017980432493) BeliefPropagation.MS{Float64}(-1.2039728043259361); BeliefPropagation.MS{Float64}(-0.6931471805599453) BeliefPropagation.MS{Float64}(0.9162907318741551)]), TabulatedBPFactor{BeliefPropagation.MS{Float64}, 2}(BeliefPropagation.MS{Float64}[BeliefPropagation.MS{Float64}(0.0) BeliefPropagation.MS{Float64}(-1.2039728043259361); BeliefPropagation.MS{Float64}(0.8329091229351039) BeliefPropagation.MS{Float64}(0.0); BeliefPropagation.MS{Float64}(-Inf) BeliefPropagation.MS{Float64}(0.1823215567939546)]), TabulatedBPFactor{BeliefPropagation.MS{Float64}, 2}(Be

Iterate the BP algorithm for `maxiter` iterations or until the maximum variation in messages between two succesive iterations is less than `tol`

In [34]:
fe = init_free_energy(bp)
iters = iterate!(bp; maxiter=10, tol=1e-12, f=fe)
println("BP converged in $iters iterations")

BP converged in 1 iterations


In [35]:
sum(fe)

-2.163323025660538

Compute single-variable marginals ("beliefs") $p(x_i)$ $\forall i$

In [36]:
b = beliefs(bp)

4-element Vector{Vector{BeliefPropagation.MS{Float64}}}:
 [BeliefPropagation.MS{Float64}(-1.3862943611198908), BeliefPropagation.MS{Float64}(-0.1823215567939546), BeliefPropagation.MS{Float64}(0.0)]
 [BeliefPropagation.MS{Float64}(-1.3404642385789354), BeliefPropagation.MS{Float64}(0.0)]
 [BeliefPropagation.MS{Float64}(-1.3404642385789354), BeliefPropagation.MS{Float64}(0.0)]
 [BeliefPropagation.MS{Float64}(-1.4582472742353187), BeliefPropagation.MS{Float64}(0.0), BeliefPropagation.MS{Float64}(-1.4213856809311607), BeliefPropagation.MS{Float64}(-0.9694005571881033)]

Compute factor marginals $p(\underline{x}_a)$ $\forall a$

In [37]:
fb = factor_beliefs(bp)

3-element Vector{Matrix{BeliefPropagation.MS{Float64}}}:
 [BeliefPropagation.MS{Float64}(-1.3404642385789354) BeliefPropagation.MS{Float64}(-2.120263536200091); BeliefPropagation.MS{Float64}(-2.1289215989432058) BeliefPropagation.MS{Float64}(0.0)]
 [BeliefPropagation.MS{Float64}(-2.1733733615140394) BeliefPropagation.MS{Float64}(-1.3862943611198908); BeliefPropagation.MS{Float64}(-1.3404642385789354) BeliefPropagation.MS{Float64}(-0.1823215567939546); BeliefPropagation.MS{Float64}(-Inf) BeliefPropagation.MS{Float64}(0.0)]
 [BeliefPropagation.MS{Float64}(-1.4582472742353187) BeliefPropagation.MS{Float64}(-Inf) BeliefPropagation.MS{Float64}(-3.5376888159151547) BeliefPropagation.MS{Float64}(-1.3404642385789354); BeliefPropagation.MS{Float64}(-Inf) BeliefPropagation.MS{Float64}(0.0) BeliefPropagation.MS{Float64}(-1.4213856809311607) BeliefPropagation.MS{Float64}(-0.9694005571881033)]

Compute the normalization constant $Z$ and compare with the result of an exhaustive computation